In [1]:
SEASON = '1996-97'

In [2]:
import pandas as pd
import myconstants as c
from scipy.stats import zscore

NUM_GAMES = c.NUM_GAMES[SEASON]
MIN_GAMES = NUM_GAMES * 0.60

p_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_PerGame.csv')
t_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Team_Standings.csv')

# join player per game and advanced stats
p_advanced_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_Advanced.csv')
p_advanced_df = p_advanced_df.drop(columns=['Player', 'Tms', 'Pos', 'Age', 'G', 'MP'])
p_df = p_df.merge(p_advanced_df, on='PlayerID')

# populate team df with team codes
t_df['TeamCode'] = t_df['Team'].apply(lambda x: c.TEAM_CODE[x])

# average team wins per player
def average_wins(tms):
    tms_list = tms.split(',')
    tms_wins = []
    for t in tms_list:
        tms_wins.append(t_df.loc[t_df['TeamCode'] == t]['OverallW'].values[0])
    return sum(tms_wins) / len(tms_wins)
p_df['TmsWins'] = p_df['Tms'].apply(average_wins)

# players who haven't played enough games don't qualify
p_df = p_df.loc[p_df['G'] >= MIN_GAMES]

# ranking algorithm
p_df['PlayerScore'] = 1.5*zscore(p_df['PTS'] + p_df['TRB'] + p_df['AST'] + p_df['STL'] + p_df['BLK']) + zscore(p_df['WS']) + zscore(p_df['VORP']) + zscore(p_df['BPM']) + zscore(p_df['TmsWins'])
p_df['PlayerScore'] = p_df['PlayerScore'] + c.PLAYER_SCORE_ADDITION # more positive PlayerScores
p_df['PlayerScore'] = p_df['PlayerScore'].apply(lambda x: pow(x, c.PLAYER_SCORE_EXPONENT) if x > 1 else x) # make higher PlayerScores more significant
p_df = p_df.sort_values('PlayerScore', ascending=False)

# Move [PlayerScore] to the right of [Pos], drop PlayerID, index rows starting from 1, stylistic choice
col = p_df.pop('PlayerScore')
p_df.insert(p_df.columns.get_loc('Pos') + 1, col.name, col)
p_df = p_df.drop('PlayerID', axis=1)
p_df = p_df.reset_index(drop=True)
p_df.index = p_df.index + 1

p_df.head(50)

,Player,Tms,Pos,PlayerScore,Age,G,GS,MP,FG,FGA,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,TmsWins
1,Michael Jordan,CHI,SG,111.876640,33,82,82,37.9,11.2,23.1,...,33.2,13.3,5.0,18.3,0.283,7.8,1.1,8.9,8.6,69.0
2,Karl Malone,UTA,PF,103.465243,33,82,82,36.6,10.5,19.2,...,32.7,11.6,5.1,16.7,0.268,7.1,1.2,8.3,7.8,64.0
3,Grant Hill,DET,SF,90.958845,24,80,80,39.3,7.8,15.7,...,28.3,8.7,5.9,14.6,0.223,5.4,2.7,8.2,8.0,54.0
4,Scottie Pippen,CHI,SF,76.351985,31,82,82,37.7,7.9,16.7,...,24.6,7.8,5.2,13.1,0.203,4.3,1.4,5.7,6.1,69.0
5,Tim Hardaway,MIA,PG,74.577717,30,81,81,38.7,7.1,17.1,...,26.6,7.9,5.0,12.9,0.198,5.2,1.0,6.2,6.5,61.0
6,John Stockton,UTA,PG,72.832405,34,82,82,35.3,5.1,9.3,...,18.4,10.0,3.6,13.6,0.226,5.6,1.0,6.6,6.3,64.0
7,Gary Payton,SEA,PG,71.777492,28,82,82,39.2,8.6,18.1,...,26.4,8.2,4.7,12.9,0.193,4.4,1.1,5.5,6.1,57.0
8,Mookie Blaylock,ATL,PG,71.351392,29,78,78,39.2,6.4,14.9,...,21.9,7.0,5.6,12.5,0.197,4.3,2.4,6.8,6.8,56.0
9,Shaquille O'Neal,LAL,C,67.040435,24,51,51,38.1,10.8,19.4,...,31.1,4.1,3.9,8.0,0.197,4.7,1.8,6.5,4.1,56.0
10,Charles Barkley,HOU,PF,61.381525,33,53,53,37.9,6.3,13.1,...,23.2,5.7,3.7,9.4,0.224,4.5,1.2,5.7,3.9,57.0
